### 简历项目1：使用DPO微调基于Qwen3-8B的推荐系统

#### 参考文献
Finetuning Large Language Model for Personalized Ranking https://arxiv.org/abs/2405.16127 <br><br>
#### 1. 前情摘要
#### 1.1 论文理解
&emsp;&emsp;在少样本或者零样本的情况下，将llm应用于推荐系统，但llm基于nlp任务设计，与推荐任务不同。SFT可以在训练过程中最大化生成正确答案的概率，当面对未见过的错误答案时，由于缺乏负样本学习，模型可能会高估其概率做出错误预测。所以，应该通过多个正负样本对之间进行学习。基于这种情况，在直接偏好优化（Direct Preference Optimization, DPO）的基础上，引入多个负样本的采样，扩大负样本的范围，建立更全面和平衡的正负样本关系。<br>

#### 1.2 BPR loss
 &emsp;&emsp;成对排序损失（Pair-Wise Ranking loss）是一种在机器学习中常用于排序任务的损失函数，它的目的是学习输入样本之间的相对距离，而不是直接预测一个标签或值，其中典型损失：贝叶斯个性化排序损失（Bayesian Personalized Ranking, BPR loss）。在推荐系统中，用户的历史行为数据通常是以隐式反馈形式存在的，例如用户的浏览、购买或点击行为。与显式反馈数据（例如用户的评分）相比，隐式反馈数据更加稀疏和难以解释。因此，推荐系统需要开发出适合隐式反馈数据的模型和算法来推荐物品。BPR损失函数就是为解决隐式反馈数据下的推荐问题而提出的。它的基本思想是：给定一个用户和两个物品，模型需要将用户更喜欢的物品排在用户更不喜欢的物品之前，从而学习到用户的个性化偏好。<br>

\begin{equation}
\mathcal{L}_{\mathrm{BPR}} = - \sum_{(u,i,j)\in\mathcal{D}} \ln \sigma\bigl(f_u(i) - f_u(j)\bigr)
\tag{1}
\end{equation}

 &emsp;&emsp; $\mathcal{D}$ 表示数据集, $u$ 表示用户, $i$ 和 $j$ 分别表示正样本物品与负样本物品。$ f_u(i)$ 表示用户 $u$ 对物品  $i$ 的偏好评分函数。常用的偏好评分函数包括余弦相似度（cosine similarity）和皮尔逊相关系数（Pearson correlation coefficient）。通过优化 BPR 损失函数，可以提高用户 $u$ 对正样本物品 $i$ 的偏好得分 $ f_u(i)$ ，同时降低其对负样本物品 $j$  的偏好得分 $ f_u(j)$。
#### 2. 总体步骤
<div align="center">
   <img src="./pics/rp1_dpo_1.png" width="800" />
</div>
 &emsp;&emsp;整个实验主要包含两个阶段，SFT微调和DPO微调，都是通过LoRA方法实现的，可以降低显存的需求。SFT和DPO所使用的用户是一样的，只不过在DPO阶段，需要加入负样本，但每个用户使用的负样本个数包含2-5，后续样本展示理解更直观，作者实验后续也做了负样本个数的消融实验。


#### 2.1 任务描述
 &emsp;&emsp;论文提出的DMPO 旨在判断用户是否喜欢所给物品，该任务本质上属于二分类任务。用户的物品偏好历史会被输入到提示中，随后从用户-物品交互列表中选取一对项目来进行预测。其中一项记为 $i$ 正样本，已被用户给予较高的评分，且高于3分（总分5分）。另一项记为负样本 $j$，被用户给予的评分较低，低于 3 分。该模型会通过生成一个推荐列表，该列表会根据用户对各项给予高评分的可能性进行排序，并将项目 $i$ 排在项目 $j$ 之前，从而确定用户更有可能喜欢正样本项目 $i$ 而非负样本项目 $j$。

#### 2.2 SFT和DPO阶段的提示词
 &emsp;&emsp;**数据集**：Amazon Datasets总共包含1.428亿条评论数据，包含书籍、电影电视、游戏。本人实践过程中只使用了作者整理的Amazon Movies and TV数据，包含，该实验是在少样本场景下进行的，训练集、验证集和测试机分别时100、100和1000。数据分为两类：小于等于3分的为负样本，大于3分的为正样本。每个用户与物品的交互数至少5个正样本和5个负样本，并且交互数量限制在40个项目以内。<br>

&emsp;&emsp; **提示词构造**：首先构建任务输入，包括角色设置、用户-物品交互列表以及正样本呢和负样本。正样本和负样本的顺序应随机排序，以防止模型根据输入生成答案。在用户-物品交互列表中，使用"<>"分割以便模型识别。在任务输出中，对于SFTT，只需要提供正确的答案"<正样本,负样本>"，正样本应排在负样本之前。对于DPO，正确和错误的答案都应该以"[<正样本,负样本>,<负样本,正样本>]"的格式给出。当有多个负样本存在时，都需要写出来。<br>

&emsp;&emsp; **样本展示**<br>
&emsp;&emsp; **SFT阶段的样本**：{"instruction": "You are an assistant working on movie recommendations. Here is the user's history of movies they have watched:  <Title:Skyfall>  <Title:R.I.P.D. Brigade fant&ocirc;me Region B  Sous-titres fran&ccedil;ais>  <Title:Warm Bodies>  <Title:World's End> . Rank the likelihood of the user watching the two movies  <Title:X-Men Origins: Wolverine>  and  <Title:The Lords Of Salem> .", "input": "", "output": " <Title:X-Men Origins: Wolverine> ,  <Title:The Lords Of Salem> "} <br>

&emsp;&emsp; **DPO阶段的样本**：

&emsp;&emsp; **(1) 只有一个负样本时**<br> 
&emsp;&emsp; {"instruction": "You are an assistant working on movie recommendations. Here is the user's history of movies they have watched:  <Title:Skyfall>  <Title:R.I.P.D. Brigade fant&ocirc;me Region B  Sous-titres fran&ccedil;ais>  <Title:Warm Bodies>  <Title:World's End> . Rank the likelihood of the user watching the two movies  <Title:X-Men Origins: Wolverine>  and  <Title:The Lords Of Salem> .", "input": "", "output": [" <Title:X-Men Origins: Wolverine> ,  <Title:The Lords Of Salem> ", " <Title:The Lords Of Salem> ,  <Title:X-Men Origins: Wolverine> "]}<br>

&emsp;&emsp; **(2) 两个负样本时**这里我自己使用函数转化了一下，作者给的数据不是这样的构造<br>
&emsp;&emsp; {
    "instruction": "You are an assistant working on movie recommendations. Here is the user's history of movies they have watched:  <Title:Skyfall>  <Title:R.I.P.D. Brigade fant&ocirc;me Region B  Sous-titres fran&ccedil;ais>  <Title:Warm Bodies>  <Title:World's End> . Rank the likelihood of the user watching the two movies  <Title:The Lords Of Salem>  and  <Title:X-Men Origins: Wolverine> .",
    "input": "",
    "chosen": " <Title:X-Men Origins: Wolverine> ,  <Title:The Lords Of Salem> ",
    "rejected": " <Title:The Lords Of Salem> ,  <Title:X-Men Origins: Wolverine> "
  }<br>
  &emsp;&emsp;{
    "instruction": "You are an assistant working on movie recommendations. Here is the user's history of movies they have watched:  <Title:Skyfall>  <Title:R.I.P.D. Brigade fant&ocirc;me Region B  Sous-titres fran&ccedil;ais>  <Title:Warm Bodies>  <Title:World's End> . Rank the likelihood of the user watching the two movies  <Title:Silent Hill: Revelation>  and  <Title:X-Men Origins: Wolverine> .",
    "input": "",
    "chosen": " <Title:X-Men Origins: Wolverine> ,  <Title:Silent Hill: Revelation> ",
    "rejected": " <Title:Silent Hill: Revelation> ,  <Title:X-Men Origins: Wolverine> "
  }<br>

&emsp;&emsp; 三个、四个、五个负样本同上，依次都列举出来。<br>
#### 2.3 SFT阶段
 &emsp;&emsp;SFT通过正样本对LLM进行微调，以完成推荐任务。其主要目标是使每个答案中的每个词的概率达到最大值。此外，SFT 还确保由语言模型生成的答案符合正确的格式，避免出现含糊不清或含糊其辞的回答。<br>

$$
\max_{\Phi} \sum_{(x,y)\in \mathcal{D}} \sum_{t=1}^{|y|} \log \big( P_{\Phi}(y_t \mid x, y_{<t}) \big),
\tag{2}
$$

&emsp;&emsp;其中，$x$ 和 $y$ 分别表示SFT中的 **“Task Input”** 和 **“Task Output”**；$y_t$ 表示输出序列 $y$ 中的第 $t$ 个 token，$y_{<t}$ 表示在 $y_t$ 之前的所有 token，$Phi$ 表示模型的原始参数，$\mathcal{D}$ 表示训练数据集。


#### 2.4 DPO阶段
&emsp;&emsp; DMPO 是基于 SFT 的，它不仅最大限度地提高了生成正确答案的概率，还降低了生成多个负面样本的概率。这有助于模型学习并建立正负样本之间复杂的关联，使其能够捕捉它们的差异和联系。此外，DMPO 还有助于抑制负面样本中关键标记词的生成概率。DMPO 中引入的多重负面采样与原始的DPO相比，扩大了采样负面样本的范围，促进了对负面样本更多样且统一的学习。这种改进最终提高了模型的性能，DMPO 的数学公式如下所示：<br>


\begin{equation}
\mathcal{L}_{\mathrm{DMPO}}(\pi_\theta;\pi_{\mathrm{ref}})
= - \mathbb{E}_{(x, y_w, y_l)\sim \mathcal{D}}
\Bigg[
\log \sigma \Bigg(
\beta \log \frac{\pi_\theta(y_w \mid x)}{\pi_{\mathrm{ref}}(y_w \mid x)}
-
\frac{1}{k} \sum_{i=1}^{k}
\Big(
\beta \log \frac{\pi_\theta(y_{l_i} \mid x_i)}{\pi_{\mathrm{ref}}(y_{l_i} \mid x_i)}
\Big)
\Bigg)
\Bigg],
\tag{3}
\end{equation}

&emsp;&emsp;其中，$k$ 表示负样本的数量，$\pi_\theta$ 表示语言模型策略，$\pi_{\mathrm{ref}}$ 表示参考策略，$x$ 表示指令输入，$\sigma$ 表示 sigmoid 激活函数，$y_w$ 和 $y_l$ 分别表示正确答案和错误答案，$\mathcal{D}$ 表示指令数据集。<br>
&emsp;&emsp; 语言模型策略 $\pi_\theta$ ：正在训练、正在被更新、希望它“学会更好排序”的那个模型。<br>
&emsp;&emsp; 参考策略 $\pi_{\mathrm{ref}}$ ：一个冻结的“基准模型”，用于告知在不经过偏好优化之前，这个回答本来有多合理？<br>
&emsp;&emsp; DPO 不直接最大化“我有多喜欢一个答案”，而是最大化“我相对于一个参考模型，更偏好这个答案多少”。


&emsp;&emsp; 在强化学习（RL）的语言里：policy（策略）= 给定状态，输出动作的概率分布。而在语言模型中,状态 = prompt（instruction + history）, 动作 = 生成下一个 token，整段输出 = 一串动作的轨迹。所以：
\begin{equation}
\pi_\theta(y \mid x)
\tag{4}
\end{equation}
表示的是：参数为 $\theta$ 的语言模型，在给定输入 $x$ 时，生成完整输出序列 $y$ 的概率。所以论文里说 $\pi_\theta$ 表示语言模型策略，本质上就是当前正在训练的LLM。 $\pi_\theta$ 被优化的对象，梯度会更新，被“拉向更偏好正排序、远离错误排序”。在 loss中，对 $y_w $：拉高 $\pi_\theta(y_w \mid x)$ ，对  $y_l $ ：压低  $\pi_\theta(y_l \mid x) $。<br>
&emsp;&emsp; 参考策略 $\pi_{\mathrm{ref}}$ 是一个冻结的、不会被更新的语言模型，在这里是SFT之后的模型。$\pi_{\mathrm{ref}}$ 的参数是固定的，不参与训练。为什么需要参考策略，如果只最大化 $\log \pi_\theta(y_w \mid x)$ 并最小化
$\log \pi_\theta(y_l \mid x)$，模型可能会：极端放大某些 token，生成非常“奇怪但自信”的输出，严重偏离原始语言能力。在 RLHF 中被称为reward hacking / mode collapse。<br>
\begin{equation}
\log \frac{\pi_\theta(y \mid x)}{\pi_{\mathrm{ref}}(y \mid x)}
\tag{5}
\end{equation}
&emsp;&emsp; 衡量当前模型相对于“原本合理的语言模型，对某个输出额外偏好（或厌恶）了多少。<br>
* 如果某个输出：
  * 在 $\pi_{\mathrm{ref}}$ 下本来就很合理
  * 在 $\pi_\theta$ 下被进一步提高，那是“安全的改进”
* 如果某个输出：
  * 在 $\pi_{\mathrm{ref}}$ 下几乎不可能
  * 但 $\pi_\theta$ 被强行推高，那是“危险偏移”<br>

&emsp;&emsp; DMPO 的核心结构是：<br>

\begin{equation}
\log \sigma \Big(
\underbrace{
\beta \log \frac{\pi_\theta(y_w \mid x)}{\pi_{\mathrm{ref}}(y_w \mid x)}
}_{\text{正排序：相对提升}}
-

\underbrace{
\frac{1}{k}\sum_i
\beta \log \frac{\pi_\theta(y_{l_i} \mid x_i)}{\pi_{\mathrm{ref}}(y_{l_i} \mid x_i)}
}_{\text{负排序：相对压制}}
\Big)
\tag{6}
\end{equation}

一句话：希望当前模型相对于参考模型，更倾向于正确排序，而更不倾向于错误排序。不是“绝对概率”，而是相对概率变化。<br>

> **参考策略不是“老师模型”**
> **也不是“oracle”**

它不负责：

* 哪个答案是对的

它只负责：

* 约束你“不要跑太远”
* 保留语言流畅性与常识

#### 2.5 LoRA技术